In [69]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt

import os

In [90]:
def simulateActivity(connectivityMatrix, simTime):
    dt = 0.1
    N = connectivityMatrix.shape[0]
    T = int(np.ceil(simTime / dt))

    activityMatrix = np.zeros((T, N))

    # Setting initial conditions
    activityMatrix[0, :] = np.random.rand(1, N)

    for i in range(1, T):
        dActivity = np.tanh(np.matmul(connectivityMatrix, activityMatrix[i-1, :])) - activityMatrix[i-1, :]
        activityMatrix[i,:] = activityMatrix[i-1,:] + dt * dActivity

    return(activityMatrix)

In [91]:
datadir = "/Users/adam2392/Documents/hhmi_connectome_estimation/TrainData/"
datafile = os.path.join(datadir, "activationMatrices.npy")
labelfile = os.path.join(datadir, "connectivityMatrices.npy")

In [148]:
dataset = np.load(datafile)
labels = np.load(labelfile)


T, N, numsamps = dataset.shape
print("T, N, numsamps: ", T, N, numsamps)

T, N, numsamps:  1000 100 10


In [149]:

class Decoder(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Decoder, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        return F.relu(self.linear2(x))


class VAE(torch.nn.Module):
    latent_dim = 8

    def __init__(self, encoder, decoder):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self._enc_mu = torch.nn.Linear(100, 8)
        self._enc_log_sigma = torch.nn.Linear(100, 8)

    def _sample_latent(self, h_enc):
        """
        Return the latent normal sample z ~ N(mu, sigma^2)
        """
        mu = self._enc_mu(h_enc)
        log_sigma = self._enc_log_sigma(h_enc)
        sigma = torch.exp(log_sigma)
        std_z = torch.from_numpy(np.random.normal(0, 1, size=sigma.size())).float()

        self.z_mean = mu
        self.z_sigma = sigma

        return mu + sigma * Variable(std_z, requires_grad=False)  # Reparameterization trick

    def forward(self, state):
#         h_enc = self.encoder(state)
#         z = self._sample_latent(h_enc)
        z = self.encoder(state)
        return self.decoder(z)


def latent_loss(z_mean, z_stddev):
    mean_sq = z_mean * z_mean
    stddev_sq = z_stddev * z_stddev
    return 0.5 * torch.mean(mean_sq + stddev_sq - torch.log(stddev_sq) - 1)

In [163]:
class Normal(object):
    def __init__(self, mu, sigma, log_sigma, v=None, r=None):
        self.mu = mu
        self.sigma = sigma  # either stdev diagonal itself, or stdev diagonal from decomposition
        self.logsigma = log_sigma
        dim = mu.get_shape()
        if v is None:
            v = torch.FloatTensor(*dim)
        if r is None:
            r = torch.FloatTensor(*dim)
        self.v = v
        self.r = r


class Encoder(torch.nn.Module):
    def __init__(self, input_size, seq_len, H, D_out):
        super(Encoder, self).__init__()
        
        input_dim = (input_size)
        
        # create GRU -> linear units
        self.rnn = torch.nn.LSTM(input_size, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        output, hidden = self.rnn(x)
        x = F.relu(self.linear2(output))
        return x


In [167]:
seq_len = T
input_dim = N
batch_size = 1

transform = transforms.Compose(
    [transforms.ToTensor()])

# define encoder
encoder_params = {
    'input_size': input_dim,
    "seq_len": T,
    "H": 100,
    "D_out": N*N
}
encoder = Encoder(**encoder_params)

# define inputs and class labels
inputs = torch.tensor(dataset[...,i].T).float()
classes = torch.tensor(labels[...,i]).flatten().float()
inputs, classes = Variable(inputs.resize_(seq_len, batch_size, input_dim)), Variable(classes)

# zero gradient of optimizer and feed in data
optimizer.zero_grad()
dec = encoder(inputs)

print(encoder)
print(inputs.shape, classes.shape)
print(dec.shape)

torch.Size([1000, 1, 100]) torch.Size([1, 1, 100])
torch.Size([1000, 1, 10000])
Encoder(
  (gru): GRU(100, 100)
  (linear2): Linear(in_features=100, out_features=10000, bias=True)
)
torch.Size([1000, 1, 100]) torch.Size([10000])
torch.Size([1000, 1, 10000])


In [162]:
decoder_params = {
    "D_in": N*N,
    "H": 10,
    "D_out": input_dim
}
decoder = Decoder(**decoder_params)
vae = VAE(encoder, decoder)

In [58]:
criterion = nn.MSELoss()
optimizer = optim.Adam(vae.parameters(), lr=0.0001)
l = None

In [59]:
print(dataset.shape, labels.shape)
numsamples = 10

(1000, 100, 10) (100, 100, 10)


In [60]:
print(vae)

VAE(
  (encoder): Encoder(
    (linear1): Linear(in_features=100000, out_features=10, bias=True)
    (linear2): Linear(in_features=10, out_features=10000, bias=True)
  )
  (decoder): Decoder(
    (linear1): Linear(in_features=10000, out_features=10, bias=True)
    (linear2): Linear(in_features=10, out_features=100000, bias=True)
  )
  (_enc_mu): Linear(in_features=100, out_features=8, bias=True)
  (_enc_log_sigma): Linear(in_features=100, out_features=8, bias=True)
)


In [62]:
for epoch in range(100):
    for i in range(numsamples):
        inputs = torch.tensor(dataset[...,i].T)
        classes = torch.tensor(labels[...,i]).flatten()
        inputs, classes = Variable(inputs.resize_(batch_size, input_dim)), Variable(classes)
        optimizer.zero_grad()
        dec = vae(inputs)
        ll = latent_loss(vae.z_mean, vae.z_sigma)
        loss = criterion(dec, inputs) + ll
        loss.backward()
        optimizer.step()
        l = loss.data[0]
        break
    break
    print(epoch, l)

# plt.imshow(vae(inputs).data[0].numpy().reshape(28, 28), cmap='gray')
# plt.show(block=True)

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'